# Datathon - Data Processing

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pandas_profiling import ProfileReport
from functools import partial
from scipy.stats.mstats import winsorize
from sklearn.cluster import KMeans
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

%matplotlib inline

# Helper Functions

### date_vars_create
- **Inputs:**
    - **df:** DataFrame to modify
    - **base_date_column:** source date column of the dataset (in our case: ofd_date)
    - **columns_to_create:** list of new date columns we want to create. Options are: 
        - weekday (e.g. 'Monday', 'Tuesday', etc.) <- dtype: object
        - month (e.g. 'January', 'February', etc.) <- dtype: object
        - day_of_month (1, 2, 3, 4, etc.) <- dtype: int64
- **Output:**
    - Modified DF with the additional chosen date columns (columns_to_create).

In [ ]:
# Change Date Vars based on column names
def date_vars_create(df, base_date_column, columns_to_create):
    df[base_date_column] = pd.to_datetime(df[base_date_column], format='%Y-%m-%d')
    df.sort_values(by=[base_date_column, 'station_code'], inplace=True)
    
    for column in columns_to_create:
        if 'weekday' == column:
            df[column] = df[base_date_column].dt.day_name()
        if 'month' == column:
            df[column] = df[base_date_column].dt.month_name()
        if 'day_of_month' == column:
            df[column] = df[base_date_column].dt.day
    
    return df

### Clustering Station Codes
- Clusters station codes to reduce cardinality.

In [ ]:
def conversion_function(value):
        return dict[value]

def encode_and_cluster_stations(df, mean_var_list, cluster_list, groupby_var, n_clusters):
    # Encode based on means
    df_grouped = pd.DataFrame(df.groupby(by=[groupby_var], as_index = False).mean())
    #df_grouped = df_grouped.drop(columns = ['country_code'])
    
    count = 0
    for variable in mean_var_list:
        count += 1 
        mean_var = df_grouped.groupby(groupby_var)[variable].mean()
        df_grouped.loc[:, ('en_var' + str(count))] = df_grouped[groupby_var].map(mean_var)
    
    df_grouped['mean_encode'] = df_grouped[mean_var_list[0]] + df_grouped[mean_var_list[1]]
    
    df_grouped.head

    # Create Cluster
    df_cluster = df_grouped[ cluster_list + ['mean_encode']]
    
    kmean = KMeans(n_clusters=n_clusters, random_state=0).fit(df_cluster)
    
    # Map DC to New Clusters
    df_mapping = list(kmean.predict(df_cluster))
    conversion_dict = {}
    
    for i in range(0, len(df_mapping)):
        conversion_dict[df_grouped[groupby_var][i]] = df_mapping[i]

    df['DC'] = df[groupby_var].apply(lambda x: conversion_dict[x])
    
    return df

### Encode Variables
- **Inputs**:
    - **df**: DataFrame to modify,
    - **var_list**: List of variables (columns) to encode. 
- Note: If fc_codes is in the list of columns to encode, it will be treated separately (i.e. not using pd.get_dummies)
- **Output**:
    - Modified DF with all dummy columns (!Original cols will be dropped!) <- dtypes: uint8

### Encode FC
- **Inputs**:
    - **df**: DataFrame to modify,
    - **col_name**: Name of the column to do special encode. (It's only fc_code, but doing this to be more flexible)
- **Output**:
    - Modified DF with dummy columns created for every fc_code (they are separated by ', ' first).

In [ ]:
def vars_encode(df, var_list):
    
    if 'fc_codes' in var_list:
        var_list = [x for x in var_list if x != 'fc_codes']
        df = encode_fc(df, 'fc_codes')
    
    prefix_list = ['DoM' if x == 'day_of_month' else x for x in var_list]
    new_data = pd.get_dummies(df, columns=var_list, prefix=prefix_list, prefix_sep='_')
    
    return new_data


def encode_fc(df, col_name):
    vals = list(df[col_name].str.split(', ').values)
    vals = [i for l in vals for i in l]
    vals = list(set(vals))
    vals.sort()

    for v in vals:
        n = col_name + '_' + v
        df[n] = df[col_name].str.contains(v)
        df[n] = df[n].astype('uint8')
    df.drop(columns=[col_name], inplace=True)
    return df

### Outlier Management
- Inputs:
    - df: DataFrame to modify,
    - column_list: List of **numeric** columns to handle outliers for. 
    - perc_limit = Outlier Threshold. Should be symmetric (i.e. top x% and bottom x% are winsorized.

In [ ]:
def outlier_management(df, column_list, perc_limit=0.025):
    for column in column_list:
        df[column] = winsorize(df[column], limits=(perc_limit, perc_limit))
    return df

### addSimpleLags_Diffs
-> This function uses a partial function (_buildLags_Diffs_) to process individual lags. We're essentially bulk applying a single lag (using map) to all columns at once).

- **Inputs**:
    - **df**: DataFrame to modify, 
    - **lag_list**: list of periods of lags or diffs to do (ex. 1 = lag by 1 row, or subtract previous row on current row),
    - **col_list**: list of columns to apply the lagging/diff(ing) to,
    - **change_choice**: either lag, lead, or diff (diff will by default be backwards diff).
- **Output**:
    - Modified DF <- dtypes: int64

In [ ]:
# Creating Lags

def addSimpleLags_Diffs(df,lag_list, col_list, change_choice = 'lag'):

    if change_choice == 'lead':
        lag_list = map(lambda x: x*(-1),lag_list)

    arr_lags = list(map(partial(_buildLags_Diffs,df=df,
                        col_list=col_list,
                        change_choice = change_choice),
                        lag_list))

    df = pd.concat([df]+arr_lags,axis = 1)

    return df

def _buildLags_Diffs(lag,df,col_list, change_choice):
    if change_choice == 'lag' or change_choice == 'lead':
        return df.groupby('station_code')[col_list].shift(lag).add_suffix(f'_{np.abs(lag)}_{change_choice}')
    elif change_choice == 'diff':
        return df.groupby('station_code')[col_list].diff(lag).add_suffix(f'_{np.abs(lag)}_{change_choice}')

### data_processor
**-> Acts like the __main__ function**

- **Inputs**:
    - **df:** DataFrame to modify,
    - **dep_vars:** list of target variables (that should not be processed but left alone.),
    - **date_col:** base date column to use for date variable creation,
    - **date_cols_ToDo:** choice of different date columns we want to create (**see date_vars_create**),
    - **lag_diff_ToDo:** choice of different lags/diff periods to do (**see addSimpleLags_Diffs**),
    - **lagChoice:** default is both (does both Lag & Diff). Otherwise will only do lag, diff, or lead. 
    - **verbose:** Bool to choose whether you want to list out all the columns created or not. 
- **Output**:
    - Final modified DF to use for Splitting + Model.

In [ ]:
def data_processor(df, dep_vars, date_col, date_cols_ToDo, univariate_flag=False, lag_ToDo=[1, 2, 4, 5, 6, 7], diff_ToDo=None, mean_vars_clustering=['OFD', 'Slam'], cluster_groupby='station_code', n_clusters=7, 
                   outlier_threshold=0.025, lagChoice='both', verbose=False):
    
    if univariate_flag:
        df['Target'] = df['Earlies_Exp'] - df['MNR_SNR_Exp']
        # All int64, including target variables.
        LagDiff_vars_ToDo = [x for x in list(df.select_dtypes(['int64']).columns)]
        
    elif not univariate_flag:
        LagDiff_vars_ToDo = [x for x in list(df.select_dtypes(['int64']).columns) if x not in dep_vars]
        

   
    # All 'object' dtype columns from the original DF (excluding the date column) should be turned into dummies
    encode_basevars_ToDo = [x for x in list(df.select_dtypes(['object']).columns) if x != date_col]
    encode_basevars_ToDo.append('DC')
    
    # Manage outliers
    df_mod = outlier_management(df, LagDiff_vars_ToDo, outlier_threshold)
    
    final_encode_list = encode_basevars_ToDo + date_cols_ToDo
    
    # Create Date Variables
    df_mod = date_vars_create(df_mod, date_col, date_cols_ToDo)
    
    if lagChoice == 'both':
        df_mod = addSimpleLags_Diffs(df_mod, lag_ToDo, LagDiff_vars_ToDo, change_choice='lag')
        df_mod = addSimpleLags_Diffs(df_mod, diff_ToDo, LagDiff_vars_ToDo, change_choice='diff')
    
    elif lagChoice in ['lag', 'diff', 'lead']:
        df_mod = addSimpleLags_Diffs(df_mod, lag_ToDo, LagDiff_vars_ToDo, change_choice=lagChoice)
        
    df_mod = df_mod.fillna(0)
    
    
    df_mod = encode_and_cluster_stations(df_mod, mean_var_list=mean_vars_clustering, cluster_list=LagDiff_vars_ToDo + dep_vars, groupby_var=cluster_groupby,  n_clusters=n_clusters)
    # Encode columns. DC should be excluded from this as we do a special encoding. 
    df_mod = vars_encode(df_mod, final_encode_list)
    
    
    if verbose:
        for column in list(df_mod.columns):
            print(column)
    else:
        print("Data processing is done.")
        print(f"We now have {len(list(df_mod.columns))} features. Woop woop.")
            
    return df_mod

# Actual Processing of Data
- Load Train Data from CSV.
- Define base parameters
- Run data processing.

- You can optionally toggle verbose arg to list out cols or not.  

In [ ]:
# Load Data
train_data = pd.read_csv('train_data.csv', sep=',')
train_data.dtypes
train_data.columns


In [ ]:
# Define Base Parameters
target_vars = ['Earlies_Exp', 'MNR_SNR_Exp']
base_date_col = 'ofd_date'
date_columns_to_create = ['weekday', 'month', 'day_of_month']

lag_to_do = []
for x in range(1, 31):
    lag_to_do.append(x)

diff_to_do = [1, 2, 3, 7]

In [ ]:
# Run data processor
final_data = data_processor(df=train_data, 
                            dep_vars=target_vars, 
                            date_col=base_date_col, 
                            date_cols_ToDo=date_columns_to_create,
                            univariate_flag=False,
                            lag_ToDo=lag_to_do,
                            diff_ToDo = diff_to_do,
                            mean_vars_clustering=['OFD', 'Slam'],
                            cluster_groupby='station_code',
                            n_clusters=7,
                            outlier_threshold=0.025,
                            lagChoice='both',
                            verbose=False)

Data processing is done.
We now have 437 features. Woop woop.
